In [1]:
import torch
print (torch.__version__)

_ = torch.manual_seed (2022)


from torch import nn

print(f'using version {torch.__version__}')

# create some known parameters
p1 = 2
p2 = -13
p3 = 26
p4 = -7
p5 = -28
p6 = 20
p7 = 1

# generate some data
def poly(x): 
    return p1*x**6 + p2*x**5 + p3*x**4 + p4*x**3 + p5*x**2 + p6*x + p7
size = 100
start = -1
end = 3
X = torch.arange(start, end, (end-start)/size)
y = poly(X) # + torch.normal(0, 0.75, size=(size,)) # if you want to add noise

# Train test split
X_train = torch.cat((X[:40], X[50:]))
y_train = torch.cat((y[:40], y[50:]))
X_test = X[40:50]
y_test = y[40:50]

# Build the model:
class PolynomialRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.p1 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p2 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p3 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p4 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p5 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True, 
                                                dtype=torch.float32))
        self.p6 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p7 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
    def forward(self, x):
        # replace self.p7*x with self.p6*x
        return self.p1*x**6 + self.p2*x**5 + self.p3*x**4 + self.p4*x**3 + self.p5*x**2 + self.p6*x + self.p7

# Create the model
torch.manual_seed(42)
model = PolynomialRegressionModel()

# Define the loss function and the optimizer
loss_fn = nn.L1Loss()
# learning_rate = 0.0001
learning_rate = 0.00001   # reduce the learning rate for stability
# add momentum to SGD to speed up training dramatically
optimizer = torch.optim.SGD(params = model.parameters(), 
                            lr = learning_rate, momentum = 0.99)

# Train the model
# epochs = 10000
epochs = 1000001   # one hundred times as many epochs
epoch_num = []
train_losses = []
test_losses = []
for epoch in range(epochs):
    model.train()
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.inference_mode():
        test_pred = model(X_test)
        test_loss = loss_fn(test_pred, y_test)

        # if epoch % 10 == 0:
        if epoch % 100000 == 0:  # print less frequently
            epoch_num.append(epoch)
            train_losses.append(loss.item())
            test_losses.append(test_loss.item())
            print(f'Epoch: {epoch} | MAE train loss: {round(loss.item(), 6)} | MAE test loss: {round(test_loss.item(), 6)}')

# compare fit polynomial coefficients to originals
print ('p1:', p1, model.p1.item())
print ('p2:', p2, model.p2.item())
print ('p3:', p3, model.p3.item())
print ('p4:', p4, model.p4.item())
print ('p5:', p5, model.p5.item())
print ('p6:', p6, model.p6.item())
print ('p7:', p7, model.p7.item())

2.3.1
using version 2.3.1
Epoch: 0 | MAE train loss: 116.280228 | MAE test loss: 1.51387
Epoch: 100000 | MAE train loss: 2.412367 | MAE test loss: 2.241235
Epoch: 200000 | MAE train loss: 2.028378 | MAE test loss: 1.833315
Epoch: 300000 | MAE train loss: 1.654884 | MAE test loss: 1.466005
Epoch: 400000 | MAE train loss: 1.283852 | MAE test loss: 1.139048
Epoch: 500000 | MAE train loss: 0.919399 | MAE test loss: 0.822977
Epoch: 600000 | MAE train loss: 0.551617 | MAE test loss: 0.488291
Epoch: 700000 | MAE train loss: 0.383948 | MAE test loss: 0.143714
Epoch: 800000 | MAE train loss: 0.273468 | MAE test loss: 0.000991
Epoch: 900000 | MAE train loss: 0.273468 | MAE test loss: 0.000991
Epoch: 1000000 | MAE train loss: 0.273468 | MAE test loss: 0.000991
p1: 2 2.0012271404266357
p2: -13 -12.998856544494629
p3: 26 26.000932693481445
p4: -7 -7.000124931335449
p5: -28 -28.000198364257812
p6: 20 20.0
p7: 1 1.0000135898590088
